# CAPM Calculations with Exponential Weights #

### CAPM Model using Weighted Linear Regression Model ###

In [29]:
# Import Libraries

# Data Management
import numpy as np
import pandas as pd

# Plots
import matplotlib.pyplot as plt

# Statistics
import statsmodels.api as sm

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from data_downloader import get_market_data

In [30]:
# Weights
def wexp(N, half_life):
    c = np.log(0.5) / half_life
    n = np.array(range(N))
    w = np.exp(c * n)
    return np.flip(w / np.sum(w))

# Create the CAPM Function
def capm_regression(
    excess_stock: pd.Series,
    excess_benchmark: pd.Series,
    window: int = 252,
    WLS: bool = False,
):
    
    X = excess_benchmark
    y = excess_stock
    
    if WLS:
        # Create weights with exponential decay
        weights = window * wexp(window, window/2)
        
        # Fit WLS regression
        model = sm.WLS(y, sm.add_constant(X), weights=weights, missing='drop').fit()
    
    else:
        # Fit OLS regression
        model = sm.OLS(y, sm.add_constant(X), missing='drop').fit()
    
    return model
    
    
def rolling_capm_regression(
    stock_returns: pd.Series,
    benchmark_returns: pd.Series,
    daily_rfr: pd.Series,
    window: int = 252,
    WLS: bool = False,
):

    #Align Data
    df = pd.concat([stock_returns, benchmark_returns, daily_rfr], axis=1)
    df = df.dropna()
    df.columns = ['stock_returns', 'benchmark_returns', 'daily_returns']

    # Compute Excess Returns
    excess_stock = df['stock_returns'] - df['daily_returns']
    excess_benchmark = df['benchmark_returns'] - df['daily_returns']

    # Lists
    alphas, betas, sigma = [], [], []
    p_values_alpha, p_values_beta = [], []
    r_squared = []
    dates = []

    for t in range(window, len(excess_stock)):
        # The variables
        X = excess_benchmark.iloc[t-window:t]
        y = excess_stock.iloc[t-window:t]

        # Create the Model
        model = capm_regression(y, X, window=window, WLS=WLS)

        # Avoid KeyError by checking if params exist
        params = model.params
        pvalues = model.pvalues
        r2 = model.rsquared
        hsigma = (model.resid).std()

        # Append values
        alphas.append(params.iloc[0])
        betas.append(params.iloc[1])
        sigma.append(hsigma)
        p_values_alpha.append(pvalues.iloc[0])
        p_values_beta.append(pvalues.iloc[1])
        r_squared.append(r2)
        dates.append(excess_stock.index[t - 1])  # Last date to calculate betas
            
    parameters = pd.DataFrame({
        'alpha': alphas,
        'beta': betas,
        'sigma': sigma,
        'p_value_alpha': p_values_alpha,
        'p_value_beta': p_values_beta,
        'r_squared': r_squared,
    }, index=pd.Index(dates, name="date"))
    
    return parameters

In [31]:
# Get the important data for the Risk-Free Rate
rfr = pd.read_csv(r"..\additional_data\rfr.csv")
rfr = rfr.set_index('Date')
rfr.index = pd.to_datetime(rfr.index)
rfr.dropna(inplace = True)

rfr

In [33]:
# Benchmark Data
data_sp500 = pd.read_csv(r'..\additional_data\sp500.csv')
data_sp500.set_index('date', inplace=True)
data_sp500.index = pd.to_datetime(data_sp500.index)

data_sp500

In [35]:
# Stock Data
ticker = 'NVDA'

df_stock = get_market_data(
        ticker=ticker, 
        start_date='2010-01-01', 
        end_date='2025-01-01', 
        returns=True
    )

df_stock

In [36]:
# Calculate the Betas using WLS
betas_wls = rolling_capm_regression(
    df_stock['returns'],
    data_sp500['sp_500'],
    rfr['risk_free_rate'],
    WLS = True
)

betas_wls

In [37]:
# Calculate the Betas using OLS
betas_ols = rolling_capm_regression(
    df_stock['returns'],
    data_sp500['sp_500'],
    rfr['risk_free_rate'],
    WLS = False,
)

betas_ols

In [38]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(betas_wls['beta'], label='WLS Beta', color='orange', alpha=0.7)
plt.plot(betas_ols['beta'], label='OLS Beta', color='blue', alpha=0.7)
plt.axhline(y=1, color='black', linestyle='dashed')

# Config
plt.title('Beta Time Series')
plt.xlabel('Time')
plt.ylabel('Beta')
plt.legend()

# Show
plt.grid()
plt.show()

In [39]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(betas_wls['alpha'], label='WLS Alpha', color='orange', alpha=0.7)
plt.plot(betas_ols['alpha'], label='OLS Alpha', color='blue', alpha=0.7)
plt.axhline(y=0, color='black', linestyle='dashed')

# Config
plt.title('Alpha Time Series')
plt.xlabel('Time')
plt.ylabel('Alpha')
plt.legend()

# Show
plt.grid()
plt.show()

In [40]:
# Create Plot
weights = 252 * wexp(252, 126)

plt.figure(figsize=(10, 6))
plt.plot(weights, label='Weights', color='black', alpha=0.7)

# Config
plt.title('Weights Graph')
plt.xlabel('Index')
plt.ylabel('Weights')
plt.legend()

# Show
plt.show()

In [41]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(betas_wls['r_squared'], label='WLS R-Squared', color='orange', alpha=0.7)
plt.plot(betas_ols['r_squared'], label='OLS R-Squared', color='blue', alpha=0.7)

# Config
plt.title('R-Squared Time Series')
plt.xlabel('Time')
plt.ylabel('R-Squared')
plt.legend()

# Show
plt.grid()
plt.show()

In [42]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(betas_wls['sigma'], label='WLS Sigma', color='orange', alpha=0.7)

# Config
plt.title('Sigma Time Series')
plt.xlabel('Time')
plt.ylabel('Sigma')
plt.legend()

# Show
plt.grid()
plt.show()

In [43]:
# Compute the Factor Contribution to Returns
def compute_factor_contributions(factor_returns, betas):
    # Multiply Elements
    contribution = (factor_returns * betas)

    return contribution

# Compute the Residual Returns
def compute_residual_returns(stock_excess_returns, factor_contributions):
    return stock_excess_returns - factor_contributions

In [44]:
# Create the Residual Returns DF
capm_returns_df = pd.DataFrame(index=betas_wls.index)

capm_returns_df['stock_excess_returns'] = df_stock['returns'] - rfr['risk_free_rate']
capm_returns_df['market_excess_returns'] = data_sp500['sp_500'] - rfr['risk_free_rate']
capm_returns_df['betas'] = betas_wls['beta']

capm_returns_df['factor_returns'] = compute_factor_contributions(
    capm_returns_df['market_excess_returns'], 
    capm_returns_df['betas']
)

capm_returns_df['residual_returns'] = compute_residual_returns(
    capm_returns_df['stock_excess_returns'],
    capm_returns_df['factor_returns']
)

capm_returns_df

In [45]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(capm_returns_df['stock_excess_returns'].cumsum(), label='Stock Returns', alpha=0.7)
plt.plot(capm_returns_df['factor_returns'].cumsum(), label='Factor Returns', alpha=0.7)
plt.plot(capm_returns_df['residual_returns'].cumsum(), label='Residual Returns', alpha=0.7)

# Config
plt.title(f'{ticker} Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()

# Show
plt.grid()
plt.show()

### Variance Decomposition ###

Variance Decomposition

1) $ \mathrm{Var}(\mu_i) = \mathrm{Var}(\beta_0 + \beta_1\mu_M + u_i) $

Distributing the Variance Operator (assuming betas are constant):

2) $ \mathrm{Var}(\mu_i) = \mathrm{Var}(\beta_0) + \mathrm{Var}(\beta_1\mu_M ) + \mathrm{Var}(u_i)$
3) $ \sigma_i^2 = \beta_1^2\sigma_M^2 + \sigma_u^2 $

Recalling the alternative calculation of the beta coefficient (check the guide in the Module 4 Files)

4) $ \sigma_i^2 = \left( \rho_{i,M}\frac{\sigma_i}{\sigma_M} \right)^2 \sigma_M^2 + \sigma_u^2 $
5) $ \sigma_i^2 = \rho_{i,M}^2\sigma_i^2 + \sigma_u^2 $

Finally, solving for the variance of the errors:

6) $ \left( 1 - \rho_{i,M}^2 \right) \sigma_i^2 = \sigma_u^2 $

In [46]:
# First Calculate the Variance of the Stock
stock_variance = df_stock['returns'].var()

stock_variance

In [47]:
# Now calculate the Variance of the Market
market_variance = data_sp500['sp_500'].var()

market_variance

In [48]:
# And the Residuals Variance
stocks_market_corr = df_stock['returns'].corr(data_sp500['sp_500'])

resid_variance = (1 - stocks_market_corr*stocks_market_corr) * stock_variance

resid_variance

In [49]:
beta_model = capm_regression(
    df_stock['returns'],
    data_sp500['sp_500'],
    rfr['risk_free_rate'],
    WLS = False,
)

print(beta_model.summary())

In [50]:
# Extract the beta
beta = beta_model.params.iloc[1]

beta

In [51]:
# Calculated Stock Variance
calculated_stock_variance = (beta*beta * market_variance) + resid_variance

calculated_stock_variance

In [54]:
# Note that the error is practically 0
(stock_variance - calculated_stock_variance).round(2)

In [53]:
# Now check how much of the variance is explained by the Market and how much by the errors

print(f'Residuals Variance Percentage: {(resid_variance / stock_variance).round(5) * 100}%')
print(f'Market Variance Percentage: {100 - ((resid_variance / stock_variance).round(5) * 100)}%')